
a demo of LR-based Avazu-CTR prediction.
    
    1.apply one-hot transform to dataset.
    2.using SGDClassifier for the training of Logistic Regression.
    3.predicting to generate submission.

In [1]:
import numpy as np
import matplotlib 
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pandas import Series,DataFrame
%matplotlib inline
matplotlib.style.use('ggplot')
import sklearn

from datetime import datetime
print(datetime.now().strftime("%Y/%m/%d %H:%M:%S"))

fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12
fig_size[1] = 8
print(np.version.full_version)
print(matplotlib.__version__)
print(pd.__version__)
print(sklearn.__version__)

2019/05/02 00:33:24
1.16.2
3.0.3
0.24.2
0.20.3


In [2]:
fp_train = "./avazu-ctr-prediction/train.csv"

fp_train_col_counts = "./avazu-ctr-prediction/col_counts"

fp_train_f = "./avazu-ctr-prediction/train_f.csv"

fp_train_lb = "./avazu-ctr-prediction/train_f_lb"
fp_train_oh = "./avazu-ctr-prediction/train_f_oh"

fp_test = "./avazu-ctr-prediction/test.csv"
fp_test_f = "./avazu-ctr-prediction/test_f.csv"



In [3]:
features_mask =  ['C1','banner_pos', 
                  'site_id', 'site_category', 
                  'app_domain', 'app_category', 
                  'device_model', 'device_type', 'device_conn_type', 
                  'C15', 'C16',  
                  'C18', 'C19', 'C20', 'C21']

#print(set(features_mask_org) - set(features_mask))
#features_mask = features_mask + ['day_week', 'hour_day']

features_unique = 'id'

target_mask = 'click'

## data reading
#date_parser = lambda x: pd.datetime.strptime(x, '%y%m%d%H')
def rawcount(filename):
    f = open(filename, 'rb')
    lines = 0
    buf_size = 1024 * 1024
    read_f = f.raw.read

    buf = read_f(buf_size)
    while buf:
        lines += buf.count(b'\n')
        buf = read_f(buf_size)

    return lines


In [4]:
import pickle  # to store temporary variable
oh_enc = pickle.load(open(fp_train_oh, 'rb'))

In [5]:
df_train_f = pd.read_csv(fp_train_f, chunksize = 500000, iterator = True,
                           dtype={'id':str}, index_col=None
                           #parse_dates=['hour'],
                           #date_parser=date_parser
                          )

In [6]:
from sklearn.linear_model import SGDClassifier  # using SGDClassifier for training incrementally
from sklearn.metrics import log_loss

lr_model = SGDClassifier(loss='log')  # using log-loss for LogisticRegression
scores = []

In [ ]:
import gc
sum = 0
for chunk in df_train_f:
    
    sum +=  len(chunk.index)
    print("～{} : {}".format(sum, datetime.now().strftime("%Y/%m/%d %H:%M:%S")))
    
    df_train = oh_enc.transform(chunk)
    
    del chunk
    
    train_X = df_train[df_train.columns.drop(['id', 'click'])]
    train_y = df_train['click'].astype('int')
    lr_model.partial_fit(train_X, train_y, classes = [0,1])  # fitting   
    
    # the score of training
    y_pred = lr_model.predict_proba(train_X)[:, 1]
    score = log_loss(train_y, y_pred)
    
    del train_X
    del train_y
    del y_pred
    del df_train
    
    gc.collect()
    
    scores.append(score)

print("end at {}".format(datetime.now().strftime("%Y/%m/%d %H:%M:%S")))

del df_train_f

～500000 : 2019/05/02 00:33:59
～1000000 : 2019/05/02 00:38:44
～1500000 : 2019/05/02 00:43:24
～2000000 : 2019/05/02 00:47:52
～2500000 : 2019/05/02 00:52:20
～3000000 : 2019/05/02 00:56:46
～3500000 : 2019/05/02 01:01:24
～4000000 : 2019/05/02 01:56:34
～4500000 : 2019/05/02 02:01:01
～5000000 : 2019/05/02 02:05:25
～5500000 : 2019/05/02 02:09:49
～6000000 : 2019/05/02 02:14:23
～6500000 : 2019/05/02 07:24:30
～7000000 : 2019/05/02 07:29:18
～7500000 : 2019/05/02 07:33:43
～8000000 : 2019/05/02 07:38:13
～8500000 : 2019/05/02 07:43:45
～9000000 : 2019/05/02 07:49:54
～9500000 : 2019/05/02 07:56:15
～10000000 : 2019/05/02 08:02:46
～10500000 : 2019/05/02 08:08:19
～11000000 : 2019/05/02 08:14:12
～11500000 : 2019/05/02 08:19:57
～12000000 : 2019/05/02 08:25:27
～12500000 : 2019/05/02 08:31:32
～13000000 : 2019/05/02 08:37:26
～13500000 : 2019/05/02 08:42:49
～14000000 : 2019/05/02 08:47:52
～14500000 : 2019/05/02 08:52:33
～15000000 : 2019/05/02 08:57:16
～15500000 : 2019/05/02 09:01:44
～16000000 : 2019/05/02 09:06

In [ ]:
## lr-model
fp_lr_model = "./avazu-ctr-prediction/lr_model"

import pickle  # to store temporary variable
## store the pre-trained lr_model
pickle.dump(lr_model, open(fp_lr_model, 'wb'))

In [ ]:
## show the training curve
f1 = plt.figure(1)
plt.plot(scores)
plt.xlabel('iterations')
plt.ylabel('log_loss')
plt.title('log_loss of training')
plt.grid()
plt.show()